<a href="https://colab.research.google.com/github/guimoll/FURB/blob/main/versao1trabalho2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import cv2, numpy as np, pandas as pd
from pathlib import Path

BASE = Path(".")
EXTS = [".png", ".jpg", ".jpeg"]

MIN_AREA_PX_NUCLEO = 90
KERNEL_OPEN  = (3,3)
GAUSS_BLUR   = (5,5)
CLAHE_CLIP   = 2.0
CLAHE_TILE   = (8,8)

# arquivos
neutros    = [f"neutrofilo{i:02d}" for i in range(10)]
linfocitos = [f"linfocito{i:02d}"  for i in range(3)]
stems = neutros + linfocitos

def resolve_path(stem: str):
    for ext in EXTS:
        p = BASE / f"{stem}{ext}"
        if p.exists(): return p
    return None

def preprocess(img_bgr):
    g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP, tileGridSize=CLAHE_TILE)
    g = clahe.apply(g)
    g = cv2.GaussianBlur(g, GAUSS_BLUR, 0)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return th

def mask_nucleos_individuais(th):
    opened = cv2.morphologyEx(
        th, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, KERNEL_OPEN), 1
    )

    dist = cv2.distanceTransform(opened, cv2.DIST_L2, 5)
    _, peaks = cv2.threshold(dist, 0.45 * dist.max(), 255, cv2.THRESH_BINARY)
    peaks = peaks.astype(np.uint8)

    num_markers, markers = cv2.connectedComponents(peaks)
    markers = markers + 1
    markers[opened == 0] = 0

    bgr = cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR)
    cv2.watershed(bgr, markers)

    mask = np.zeros_like(opened, np.uint8)
    mask[markers > 1] = 255
    return mask

def contar_componentes(mask, min_area):
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    count = 0
    for lab in range(1, num):
        if stats[lab, cv2.CC_STAT_AREA] >= min_area:
            count += 1
    return count

rows = []
for stem in stems:
    p = resolve_path(stem)
    if p is None:
        continue
    img = cv2.imread(str(p))
    if img is None:
        continue

    th = preprocess(img)
    mask = mask_nucleos_individuais(th)
    cnt  = contar_componentes(mask, MIN_AREA_PX_NUCLEO)
    rows.append({"imagem": p.name, "contagem_nucleos": int(cnt)})

df = pd.DataFrame(rows).sort_values("imagem").reset_index(drop=True)
print(df.to_string(index=False))


          imagem  contagem_nucleos
 linfocito00.png                 4
 linfocito01.png                 3
 linfocito02.png                 1
neutrofilo00.png                 4
neutrofilo01.png                 3
neutrofilo02.png                 3
neutrofilo03.png                 2
neutrofilo04.png                 4
neutrofilo05.png                 3
neutrofilo06.png                 4
neutrofilo07.png                 2
neutrofilo08.png                 2
neutrofilo09.png                 1
